In [ ]:
import tensorflow as tf
from transformers import pipeline
import json
import sys

# --- Cargar el Modelo Pre-entrenado ---
# Usamos 'pipeline' de Hugging Face, que simplifica mucho el uso de modelos pre-entrenados.
# Este modelo clasifica el texto en las siguientes emociones:
# LABEL_0: sadness (Tristeza)
# LABEL_1: joy (Alegría)
# LABEL_2: love (Amor)
# LABEL_3: anger (Enojo)
# LABEL_4: fear (Miedo)
# LABEL_5: surprise (Sorpresa)

try:
    # Intentamos cargar el modelo especificando TensorFlow como el framework.
    # Aseguramos que se usen los pesos del modelo compatibles con TF.
    emotion_classifier = pipeline(
        "sentiment-analysis",
        model="bhadresh-savani/bert-base-uncased-emotion",
        tokenizer="bhadresh-savani/bert-base-uncased-emotion",
        framework="tf" # Forzamos el uso de TensorFlow
    )
    print("Modelo de TensorFlow para clasificación de emociones cargado exitosamente.", file=sys.stderr)
except Exception as e:
    print(f"Error cargando el modelo de TensorFlow: {e}.", file=sys.stderr)
    print("Esto puede ocurrir si TensorFlow no está configurado correctamente o si el modelo prefiere PyTorch.", file=sys.stderr)
    print("Intentando cargar el modelo con el framework predeterminado (generalmente PyTorch como fallback).", file=sys.stderr)
    # Como fallback, si hay un problema con la configuración de TF, intenta cargar sin especificar framework.
    # El pipeline intentará encontrar un backend compatible (a menudo PyTorch).
    emotion_classifier = pipeline(
        "sentiment-analysis",
        model="bhadresh-savani/bert-base-uncased-emotion",
        tokenizer="bhadresh-savani/bert-base-uncased-emotion",
    )
    print("Modelo cargado con el framework predeterminado (posiblemente PyTorch).", file=sys.stderr)


def analyze_text_sentiment(text_paragraph):
    """
    Analiza los sentimientos (emociones) en un párrafo de texto usando un modelo pre-entrenado.

    Args:
        text_paragraph (str): El párrafo de texto a analizar.

    Returns:
        dict: Un diccionario con el sentimiento principal detectado, su confianza
              y, si aplica, un mensaje de error o de texto vacío.
              Ej: {"sentimiento": "Alegría", "confianza": 0.9876}
    """
    if not text_paragraph or not text_paragraph.strip():
        return {"sentimiento": "neutral", "confianza": 1.0, "mensaje": "Texto de entrada vacío o solo espacios."}

    try:
        # El pipeline devuelve una lista de diccionarios, tomamos el primer resultado (el más relevante).
        results = emotion_classifier(text_paragraph)[0]

        # Mapeamos las etiquetas internas del modelo a nombres de emociones en español
        label_map = {
            "LABEL_0": "Tristeza",
            "LABEL_1": "Alegría",
            "LABEL_2": "Amor",
            "LABEL_3": "Enojo",
            "LABEL_4": "Miedo",
            "LABEL_5": "Sorpresa"
        }

        # Obtenemos el sentimiento principal y lo mapeamos a español
        main_sentiment = label_map.get(results["label"], "Desconocido")
        # Redondeamos la confianza para una mejor presentación
        confidence = round(float(results["score"]), 4)

        return {"sentimiento": main_sentiment, "confianza": confidence}

    except Exception as e:
        print(f"Error inesperado durante el análisis de sentimiento: {e}", file=sys.stderr)
        return {"sentimiento": "error", "confianza": 0.0, "mensaje": f"Error interno del modelo: {e}"}





if __name__ == "__main__":
    print("\n--- Analizador de Sentimientos Interactivo con TensorFlow ---")
    print("Modelo: bhadresh-savani/bert-base-uncased-emotion (Clasificación de emociones)")
    print("Escribe un párrafo o frase. Escribe 'salir' para terminar.")
    print("-" * 80)

    while True:
        try:
            user_input = input("\nIntroduce tu texto aquí: ")
            if user_input.lower() == 'salir':
                print("¡Hasta luego!")
                break
            
            sentiment_output = analyze_text_sentiment(user_input)
            
            # Imprimimos el resultado de forma legible
            print("\n--- Resultado del Análisis ---")
            print(f"Sentimiento: {sentiment_output['sentimiento']}")
            print(f"Confianza: {sentiment_output['confianza'] * 100:.2f}%")
            if 'mensaje' in sentiment_output:
                print(f"Mensaje: {sentiment_output['mensaje']}")
            print("------------------------------")
            
        except Exception as e:
            print(f"Ocurrió un error inesperado: {e}")
            print("Por favor, inténtalo de nuevo o escribe 'salir'.")